# 01. Import Libraries

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

# 02. Import Data

In [2]:
path = r"C:\Users\edriesberg\Google Drive\Instacart Basket Analysis"

In [3]:
ords_prods_merge = pd.read_pickle(os.path.join(path,'02 Data','Prepared Data','orders_products_new_vars.pkl'))

# 03. Add Order Flag

In [4]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['user_order_number'].transform(np.max)

In [5]:
ords_prods_merge.loc[ords_prods_merge['max_order']>40,'loyalty_flag'] = 'Loyal customer'

In [6]:
ords_prods_merge.loc[(ords_prods_merge['max_order']<=40)&(ords_prods_merge['max_order']>10),'loyalty_flag'] = 'Regular customer'

In [7]:
ords_prods_merge.loc[ords_prods_merge['max_order']<=10, 'loyalty_flag'] = 'New customer'

# 04. Exercise

## Question 2.
In this Exercise, you learned how to find the aggregated mean of the “order_number” column grouped by “department_id” for a subset of your dataframe. Now, repeat this process for the entire dataframe.

In [8]:
ords_prods_merge.groupby('department_id')['user_order_number'].mean()

department_id
1     15.457838
2     17.277920
3     17.170395
4     17.811403
5     15.215751
6     16.439806
7     17.225802
8     15.340650
9     15.895474
10    20.197148
11    16.170638
12    15.887671
13    16.583536
14    16.773669
15    16.165037
16    17.665606
17    15.694469
18    19.310397
19    17.177343
20    16.473447
21    22.902379
Name: user_order_number, dtype: float64

## Question 3.
Analyze the result. How do the results for the entire dataframe differ from those of the subset? Include your comments in a markdown cell below the executed code.

The deparment with the highest number of average orders is `deli` with 22.9 orders on average per customer.

## Question 4.
Follow the instructions in the Exercise for creating a loyalty flag for existing customers using the transform() and loc() functions.

See Section 3 for this process.

## Question 5.
The marketing team at Instacart wants to know whether there’s a difference between the spending habits of the three types of customers you identified. Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer). What you’re trying to determine is whether the prices of products purchased by loyal customers differ from those purchased by regular or new customers.

In [9]:
ords_prods_merge.groupby('loyalty_flag').agg({'prices':['mean','median','min','max']})

prices                     
                       mean median  min      max
loyalty_flag                                    
Loyal customer    10.386336    7.4  1.0  99999.0
New customer      13.294670    7.4  1.0  99999.0
Regular customer  12.495717    7.4  1.0  99999.0

## Question 6.
The team now wants to target different types of spenders in their marketing campaigns. This can be achieved by looking at the prices of the items people are buying. Create a spending flag for each user based on the average price across all their orders using the following criteria:

- If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”

- If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”

In [10]:
ords_prods_merge['avg_spend'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.mean)

In [11]:
ords_prods_merge[['user_id','avg_spend']].head(50)

,user_id,avg_spend
0,1,6.367797
1,1,6.367797
2,1,6.367797
3,1,6.367797
4,1,6.367797
5,1,6.367797
6,1,6.367797
7,1,6.367797
8,1,6.367797
9,1,6.367797


In [12]:
ords_prods_merge.loc[ords_prods_merge['avg_spend']<10,'spend_flag'] = 'Low spender'

In [13]:
ords_prods_merge.loc[ords_prods_merge['avg_spend']>=10,'spend_flag'] = 'High spender'

In [14]:
ords_prods_merge[['user_id','avg_spend','spend_flag']].head(100)

,user_id,avg_spend,spend_flag
0,1,6.367797,Low spender
1,1,6.367797,Low spender
2,1,6.367797,Low spender
3,1,6.367797,Low spender
4,1,6.367797,Low spender
...,...,...,...
95,360,10.006250,High spender
96,377,8.496552,Low spender
97,387,7.396610,Low spender
98,420,7.387805,Low spender


## Question 7.
In order to send relevant notifications to users within the app (for instance, asking users if they want to buy the same item again), the Instacart team wants you to determine frequent versus non-frequent customers. Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
- If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
- If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
- If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

In [15]:
# Somehow, my days_since_prior_order still has NA values, so that needs to be fixed first to make NaN == 0

ords_prods_merge['days_since_prior_order'] = ords_prods_merge['days_since_prior_order'].fillna(0)

In [16]:
ords_prods_merge.head(10)

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,prices,_merge,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_spend,spend_flag
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,9.0,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender


In [17]:
ords_prods_merge['order_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

In [18]:
ords_prods_merge.head(20)

,order_id,user_id,user_order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,...,_merge,price_range_loc,busiest day,busiest days,busiest_period_of_day,max_order,loyalty_flag,avg_spend,spend_flag,order_freq
0,2539329,1,1,2,8,0.0,196,1,0,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.0
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.0
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.0
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,6.367797,Low spender,20.0
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,6.367797,Low spender,20.0
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender,20.0
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender,20.0
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,both,Mid-range product,Regularly busy,Most busy,Most orders,10,New customer,6.367797,Low spender,20.0
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,6.367797,Low spender,20.0


In [19]:
ords_prods_merge.loc[ords_prods_merge['order_freq']>20,'order_freq_flag'] = 'Non-frequent customer'

In [20]:
ords_prods_merge.loc[(ords_prods_merge['order_freq']>10) & (ords_prods_merge['order_freq']<=20),'order_freq_flag'] = 'Regular customer'

In [21]:
ords_prods_merge.loc[ords_prods_merge['order_freq']<=10,'order_freq_flag'] = 'Frequent customer'

In [22]:
ords_prods_merge.to_pickle(os.path.join(path,'02 Data','Prepared Data','ords_prods_merge.pkl'))